In [1]:
import pandas as pd
import numpy as np

import os

def load_hotel_reserve():
  customer_tb = pd.read_csv('./data/customer.csv')
  hotel_tb = pd.read_csv('./data/hotel.csv')
  reserve_tb = pd.read_csv('./data/reserve.csv')
  return customer_tb, hotel_tb, reserve_tb


def load_holiday_mst():
  holiday_tb = pd.read_csv('./data/holiday_mst.csv',
                           index_col=False)
  return holiday_tb


def load_production():
  production_tb = pd.read_csv('./data/production.csv')
  return production_tb


def load_production_missing_num():
  production_tb = pd.read_csv('./data/production_missing_num.csv')
  return production_tb


def load_production_missing_category():
  production_tb = pd.read_csv('./data/production_missing_category.csv')
  return production_tb


def load_monthly_index():
  monthly_index_tb = \
    pd.read_csv('./data/monthly_index.csv')
  return monthly_index_tb


def load_meros_txt():
  with open('./data/txt/meros.txt', 'r') as f:
    meros = f.read()
    f.close()
  return meros


In [2]:
production_tb = load_production()

#### 4. 학습용과 검증용 데이터 나누기

개요

-> 데이터 분할은 예측 모델을 평가할 때 필요한 전처리

-> 주로 학습 데이터와 검증 데이터 분할에 사용

-> 학습 데이터와 검증 데이터는 같은 전처리를 적용하며, 되도록 같은 데이터로 묶어서 다루고,
예측모델에 입력하기 직전 분할하는 것이 적절

-> 적용데이터의 경우 답을 알 수 없는 상태에서 사용하는 데이터로 데이터를 얻는 방법과 흐름, 타이밍도 다름

-> 따라서 적용 데이터의 경우 데이터를 분할할 필요가 없음

- 머신러닝 검증을 위한 데이터 레코드 분할  

    -> 교차 검증: 데이터를 검증용 데이터와 학습용 데이터로 구분

    -> 홀드아웃 검증: 교차 검증용 데이터와 별개로 최종 정확도 검증을 위한 데이터를 미리 준비 후 검증



In [5]:
# 제조 레코드를 사용하여 예측 모델 구축을 위한 데이터 분할 (교차검증)
production_tb = load_production()
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

train_data, test_data, train_target, test_target = \
    train_test_split(production_tb.drop('fault_flg', axis=1),
                     production_tb[['fault_flg']],
                     test_size=0.2)

train_data.reset_index(inplace=True, drop=True)
test_data.reset_index(inplace=True, drop=True)
train_target.reset_index(inplace=True, drop=True)
test_target.reset_index(inplace=True, drop=True)

row_no_list = list(range(len(train_target)))

k_fold = KFold(n_splits=4, shuffle=True)

for train_cv_no, test_cv_no in k_fold.split(row_no_list):
    train_cv = train_data.iloc[train_cv_no, :]
    test_cv = train_data.iloc[test_cv_no, :]
    print("Train set (head 3): ")
    print(train_cv.head(3))
    print("\nTest set (head 3): ")
    print(test_cv.head(3))

Train set (head 3): 
  type      length  thickness
0    A   76.983014   1.506188
1    C  173.731398   9.768606
2    C  287.711223   1.560784

Test set (head 3): 
   type      length  thickness
3     C  336.398100  36.188179
4     A  129.015803   9.712732
14    E  229.053544  27.472758
Train set (head 3): 
  type      length  thickness
0    A   76.983014   1.506188
1    C  173.731398   9.768606
2    C  287.711223   1.560784

Test set (head 3): 
   type      length  thickness
7     D  107.640988   9.614387
11    A  208.423057  13.938914
18    E  287.298333  47.547817
Train set (head 3): 
  type      length  thickness
2    C  287.711223   1.560784
3    C  336.398100  36.188179
4    A  129.015803   9.712732

Test set (head 3): 
  type      length  thickness
0    A   76.983014   1.506188
1    C  173.731398   9.768606
5    A  153.125043  15.585476
Train set (head 3): 
  type      length  thickness
0    A   76.983014   1.506188
1    C  173.731398   9.768606
3    C  336.398100  36.188179

Test

In [6]:
# 교차 검증 시 시간 데이터를 이용할 경우 과거 데이터로 학습하고 미래 데이터로 검증
monthly_index_tb = load_monthly_index()

train_window_start = 1
train_window_end = 24
horizon = 12
skip = 12

monthly_index_tb.sort_values(by='year_month')

while True:
    test_window_end = train_window_end + horizon
    train = monthly_index_tb[train_window_start:train_window_end]
    test = monthly_index_tb[(train_window_end + 1): test_window_end]
    if test_window_end >= len(monthly_index_tb.index):
        break

    print(train_window_start)
    print(train_window_end)
    train_window_start += skip
    train_window_end += skip

1
24
13
36
25
48
37
60
49
72
61
84
73
96
